# 계산 통계
## 1. 분포

>- 통계에서 분포는 어떤 값과 그 값이 나타날 확률의 집합
>- 확률 질량 함수(probability mass function)으로 분포를 수학적으로 표현 가능
>- thinkbayes 모듈은 다음 [링크](http://thinkbayes.com/thinkbayes.py)에서 다운 가능

In [1]:
from thinkbayes import Pmf

In [2]:
pmf = Pmf()
for x in range(1,7):
    pmf.Set(x, 1/6)

- Pmf는 아무 값도 부여되지 않은 텅 빈 Pmf 객체를 생성
- Set 메서드는 각 값에 확률 값을 설정 (위의 예제에서는 주사위 확률 부여)

In [3]:
pmf = Pmf()
for word in word_list:
    pmf.Incr(word, 1)

NameError: name 'word_list' is not defined

- Incr 메서드는 word_list의 word마다 '확률'을 1씩 높힘 (정규화 되지 않은 확률)

In [ ]:
pmf.Normalize()

- 위 방식으로 모든 빈도 수집 시 Normalize 메서드롤 확률 계산

In [ ]:
print(pmf.Prob('the'))

- Prob 메서드로 특정 값의 빈도 출력 가능
___

## 2. 쿠키문제

- 앞서 보았던 쿠키 문제를 Pmf로 해석

In [4]:
pmf = Pmf()

pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

- Set으로 사전 분포를 설정

In [5]:
pmf.Prob('Bowl 1')

0.5

In [6]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)
pmf.Prob('Bowl 1')

0.375

- 그릇 1에서 바닐라 쿠키를 선택할 우도(확률)은 3/4, 그릇 2에서 바닐라쿠키를 선택할 우도(확률)은 1/2임
- 위 정보를 바탕으로 Mult 메서드로 분포 갱신
- 단, 위와 같이 갱신 시 위 가설은 상호 배반적이며, 전체 포괄적임 -> __정규화 필요__

In [7]:
pmf.Normalize()
pmf.Prob('Bowl 1')

0.6000000000000001

- 각 가설에 대한 사후 확률을 포함하는 사후 분포로 나타남
- 그릇 1의 사후확률은 0.6임

In [8]:
pmf.GetDict()

{'Bowl 1': 0.6000000000000001, 'Bowl 2': 0.4}

___
## 3. 베이지안 프레임워크
- 앞 장에서 다룬 코드를 클래스로 정의

In [10]:
class Cookie(Pmf):
    
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

- __init__ 메서드는 각 가설별로 동일한 사전확률을 부여


In [11]:
hypos = ['Bowl 1', 'Bowl 2']
pmf = Cookie(hypos)

In [14]:
pmf.GetDict()

{'Bowl 1': 0.5, 'Bowl 2': 0.5}